# HISSO Logging CLI Walkthrough (CPU+GPU)

This walkthrough covers CPU validation and the finalized GPU sweep results for WaveResNet on Runpod L4.

## Goals

- Run the HISSO logging CLI end-to-end on CPU so we can validate configs and artifact layouts locally.
- Capture the exact command line, expected outputs, and troubleshooting tips for the Colab companion copy.
- Leave clearly marked placeholders where GPU-derived metrics and screenshots will be inserted after the accelerator run.

## Environment prep

1. Clone the repository (or install the `psann` wheel once the cleanup branch ships).
2. Ensure the datasets referenced by your chosen config are staged; the smoke templates expect CPU-scale subsets under `datasets/`.
3. Activate the project virtual environment (`python -m venv .venv && .venv\\Scripts\\activate` on Windows, `source .venv/bin/activate` elsewhere).
4. Install dependencies: `pip install -e .[dev]`.

In [ ]:
# Example CPU smoke run (replace output directory as needed)
from datetime import datetime

run_name = f"dense_cpu_smoke_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

!python -m psann.scripts.hisso_log_run \
    --config configs/hisso/dense_cpu_smoke.yaml \
    --output-dir runs/hisso/dense_cpu_smoke_dev \
    --device cpu \
    --run-name {run_name} \
    --seed 7

## Expected outputs

- `metrics.json`: Summaries (loss curves, reward aggregates, throughput).
- `events.csv`: Per-epoch log including optimiser state, dataloader timings, and shuffle policy.
- `checkpoints/`: Best estimator checkpoint plus optional intermediate snapshots when `--keep-checkpoints` is provided.
- `config_resolved.yaml`: Fully materialised config for reproducibility notes.

> GPU sweep (Runpod L4, WaveResNet small)
>
> `runs/hisso/wave_resnet_cuda_runpod_20251102_212855/` — duration ~18.37 s, throughput ~113.07 eps/s; best_epoch 56; train/val/test 0.722 / 0.864 / 0.835; reward_mean −0.114 (±0.0103); turnover 3.18; AMP float16.


## Colab companion

1. Open this notebook in Google Colab using the badge in `notebooks/README.md`.
2. Run the setup cell below (adds `psann` editable install and any config deps).
3. Mount Drive (or point `OUTPUT_ROOT` to `/content`) so artifacts persist between sessions.
4. Execute the CLI command cell; GPU timing reference (L4): ~18–19 s for 1920 episodes, ~107–113 eps/s (AMP float16).

In [ ]:
# Colab/Runpod setup + GPU run example (WaveResNet)
# 1) Install package (editable install shown)
!pip install -e .[dev]

# 2) Run HISSO logging CLI on GPU (AMP float16 enabled by config)
!python -m psann.scripts.hisso_log_run \
    --config configs/hisso/wave_resnet_small.yaml \
    --output-dir runs/hisso/wave_resnet_cuda_runpod_2025 \
    --device cuda:0 \
    --run-name wave_resnet_gpu \
    --seed 42 \
    --keep-checkpoints
